In [24]:
from functools import wraps, partial
import inspect

# 一 (不行，因为会返回一个函数而不是类，类变量无法访问，而且多次 init)

In [2]:
def singleton(cls): 
    assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'
    
    instances = {}

    def _singleton(*args, **kw):
        if cls not in instances:
            instances[cls] = cls(*args, **kw)
        return instances[cls]

    return _singleton

# 二 (可以，返回一个类，类变量仍可以访问)

In [89]:
def Singleton(cls):
    # 如果装饰在基类上, 则继承类的实例的类变量永远是基类的类变量, 而不能拥有自己的类变量
    assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

    def run_only_once(func):
        """
         使__init__只运行一次
        """

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                
                return func(self, *args, **kwargs)

        return _wrapper_init

    def newfunc(clsobj, *args, **kwargs):
        """
        使用元编程实现单例
        """
        if not hasattr(clsobj, "__instance"):
            setattr(clsobj, "__instance", object.__new__(clsobj))
        return getattr(clsobj, "__instance")

    # 拦截__new__方法
    setattr(cls, "__new__", classmethod(newfunc))
    # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
    # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
    # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
    # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
    setattr(cls, "__init__", run_only_once(cls.__init__))
    return cls

## 测试

In [91]:
tt = 0

def timer():
    global tt
    tt += 1
    return tt

@Singleton  
# @singleton  
class Person:
#     t = timer()
    a = 1

    def __init__(self, name=None, age=None):
        print("__init__!")
        self.name = name
        self.age = age
#         self.t = Person.t

def create_person():
    person1 = Person('lucky', 24)
    person2 = Person('lau', 26)
    person3 = Person()
    print(id(person1))
    print(id(person2))
    print(id(person3))

create_person()

__init__!
139947553174192
139947553174192
139947553174192


In [92]:
class People(Person):
    a = 2
    
p = People()
print(p.a)

1


# 三 (不行，因为会返回一个函数而不是类，类变量无法访问)

In [5]:
def singleton_by_features(**features):
    instances = {}  # { <features_dict_frozenset>: <instance> }
    fts = set(features.items())
    fts_len = len(fts)
    fts_frozen = frozenset(features.items())
    def singleton(cls):
        assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'
        
        def _singleton(*args, **kw):
            instanced = fts_frozen in instances
            if not instanced or (instanced and len(fts - set(kw.items())) == fts_len):  # 说明没有任何交集
                instances[fts_frozen] = cls(*args, **kw)  # 则实例化
            return instances[fts_frozen]  # 否则返回单例

        return _singleton

    return singleton

# 四 (不行，因为会返回一个函数而不是类，类变量无法访问)

In [6]:
def singleton_over_features(*features):
    instances = {}  # { <features_dict_frozenset>: <instance> }
    print("meow")
    def singleton(cls):
        assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

        def _singleton(*args, **kw):
            fts = {k: v for k, v in kw.items() if k in features}
            fts_set = set(fts.items())
            fts_frozenset = frozenset(fts.items())

            instanced = fts_frozenset in instances
            if not instanced or (instanced and len(fts_set - set(kw.items())) == len(fts_set)):  # 说明没有任何交集
                instances[fts_frozenset] = cls(*args, **kw)  # 则实例化
            return instances[fts_frozenset]  # 否则返回单例

        return _singleton

    return singleton

## 测试

In [7]:
class A:
    def __init__(self, a, b=None):
        print("[A]__init__!")
        self.a = a
        self.b = 2

# @singleton_by_features(a=1)
@singleton_over_features('a', 'b')
class B:
    def __init__(self, a, b=None):
        print("[B]__init__!")
        self.a = a
        self.b = 2
    
a1 = A(1)
a2 = A(1)
b1 = B(a=a1,b=2)
b2 = B(a=a2,b=2)
print(id(a1))
print(id(a2))
print(id(b1))
print(id(b2))

meow
[A]__init__!
[A]__init__!
[B]__init__!
[B]__init__!
139947553841448
139947553841728
139947553841840
139947553841560


# 五 (可以, 基于特征的单例, 不可以, 因为 list is unhashable)

In [13]:
def singleton_on_features(*features):

    instances = {}  # { <features_dict_frozenset>: <instance> }

    def singleton(cls):
        assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

        func = cls.__init__

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                return func(self, *args, **kwargs)
#             else:
#                 raise Exception("Conflict naming: ___init__called")

        def _new(clsobj, *args, **kwargs):
            """
            使用元编程实现单例
            """
            fts = {k: v for k, v in kwargs.items() if k in features}
            fts_set = set(fts.items())
            fts_frozenset = frozenset(fts.items())

            instanced = fts_frozenset in instances
            if not instanced or (instanced and len(fts_set - set(kwargs.items())) == len(fts_set)):  # 说明没有任何交集
                instances[fts_frozenset] = object.__new__(clsobj)  # 则实例化
            return instances[fts_frozenset]  # 否则返回单例

        # 拦截__new__方法
        setattr(cls, "__new__", classmethod(_new))
        # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
        # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
        # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
        # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
        setattr(cls, "__init__", _wrapper_init)
        return cls

    return singleton

In [14]:
class C:
    def __init__(self, a, b=None):
        print("[C]__init__!")
        self.a = a
        self.b = 2

@singleton_on_features('a', 'b')
class D:
    x = 'x'
    def __init__(self, a, b=None):
        print("[D]__init__!")
        self.a = a
        self.b = 2
        print(D.x)
        
c1 = C(1)
c2 = C(1)
d1 = D(a=a1,b=2)
d2 = D(a=a1,b=2)
print(id(c1))
print(id(c2))
print(id(d1))
print(id(d2))
print(d2.x)

[C]__init__!
[C]__init__!
[D]__init__!
x
139947553356824
139947553356600
139947553353856
139947553353856
x


# 六 (来自 interview-python)

In [ ]:
class Singleton(object):
    def __new__(cls, *args, **kw):
        if not hasattr(cls, '_instance'):
            orig = super(Singleton, cls)
            cls._instance = orig.__new__(cls, *args, **kw)
        return cls._instance

class MyClass(Singleton):
    a = 1
    def __init__(self):  # 这个是我额外加的, 显示了 虽然单例但是 init 多次
        print("__init__ !")

In [ ]:
my = MyClass()
my_too = MyClass()
my == my_too

# 七

In [21]:
def singleton_on_features(*features):
    
    instances = {}  # { <inst_key>: <instance> }

    def singleton(cls):
        assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

        func = cls.__init__

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                return func(self, *args, **kwargs)
#             else:
#                 raise Exception("Conflict naming: ___init__called")

        def _get_instance_key(fts, kw):
            for ftk, ftv in fts.items():
                try:
                    if kw[ftk] != ftv:
                        print("not matched")
                        return None
                except KeyError:
                    continue
            print("matched")
            return frozenset(fts.keys())

        def _new(clsobj, *args, **kwargs):
            """
            使用元编程实现单例
            """
            kw = {k: v for k, v in kwargs.items() if k in features}
            key = frozenset(kw.keys())
            
            if not instances.get(key):
                inst_key = partial()
                instances[inst_key] = object.__new__(clsobj)  # 则实例化
            return instances[inst_key]  # 否则返回单例

        # 拦截__new__方法
        setattr(cls, "__new__", classmethod(_new))
        # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
        # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
        # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
        # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
        setattr(cls, "__init__", _wrapper_init)
        return cls

    return singleton

In [23]:
class C:
    def __init__(self, a, b=None):
        print("[C]__init__!")
        self.a = a
        self.b = 2

@singleton_on_features('a', 'b')
class D:
    x = 'x'
    def __init__(self, a, b=None):
        print("[D]__init__!")
        self.a = a
        self.b = 2
        print(D.x)
        
c1 = C(1)
c2 = C(1)
d1 = D(a=a1,b=2)
d2 = D(a=a1,b={})
print(id(c1))
print(id(c2))
print(id(d1))
print(id(d2))
print(d2.x)

[C]__init__!
[C]__init__!
[D]__init__!
x


TypeError: unhashable type: 'dict'

# 八 单特征

In [51]:
def singleton_on_feature(feature):  # 注: 暂 feature_value 不支持 dict 和 list
    
    instances = {}  # { <inst_key>: <instance> }

    def singleton(cls):
        assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

        func = cls.__init__

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                return func(self, *args, **kwargs)

        def _new(clsobj, *args, **kwargs):
            """
            使用元编程实现单例
            """
            
            try:
                feature_value = kwargs[feature]
                inst_key = (feature, feature_value)  # 构造 key
            except:
                inst_key = None
                
            if not inst_key or not instances.get(inst_key):  # 没有 inst_key 或有但是没有对应实例
                instances[inst_key] = object.__new__(clsobj)  # 则实例化并保存
    
            return instances[inst_key]  # 返回单例

        # 拦截__new__方法
        setattr(cls, "__new__", classmethod(_new))
        # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
        # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
        # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
        # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
        setattr(cls, "__init__", _wrapper_init)
        return cls

    return singleton

In [50]:
class C:
    def __init__(self, a, b=None):
        print("[C]__init__!")
        self.a = a
        self.b = 2

@singleton_on_feature('b')
class D:
    x = 'x'
    def __init__(self, a, b=None):
        print("[D]__init__!")
        self.a = a
        self.b = 2
        print(D.x)
        
c1 = C(1)
c2 = C(1)
d1 = D(a=c1,b=(1,))
d2 = D(a=c1,b=(1,))
print(id(c1))
print(id(c2))
print(id(d1))
print(id(d2))
print(d2.x)

[C]__init__!
[C]__init__!
[D]__init__!
x
139947553085424
139947553085872
139947553085480
139947553085480
x


## 九 修正基类问题

In [110]:
def SingletonX(cls):
    # 如果装饰在基类上, 则继承类的实例的类变量永远是基类的类变量, 而不能拥有自己的类变量
    assert inspect.isclass(cls), 'The Singleton decorator can only be applied to class'

    def run_only_once(func):
        """
         使__init__只运行一次
        """

        @wraps(func)
        def _wrapper_init(self, *args, **kwargs):
            if not hasattr(self, "___init__called"):
                self.___init__called = True
                print("meow: %s" % self.a)
                return func(self, *args, **kwargs)

        return _wrapper_init

    def newfunc(clsobj, *args, **kwargs):
        """
        使用元编程实现单例
        """
        if not hasattr(clsobj, "__instance"):
            setattr(clsobj, "__instance", object.__new__(clsobj))
        return getattr(clsobj, "__instance")

    # 拦截__new__方法
    setattr(cls, "__new__", newfunc)
    # 由于在创建cls时，python会调用__new__来创建实例，然后再调用实例的__init__
    # 这样通过在__new__使用object__new__能确保每次实例化cls时均返回同一个实例。
    # 但是python还是会调用__new__的__init__，这样会导致__init__方法每实例一次就会调用一次
    # 这种行为不是我们需要的，因此我们通过run_on_once装饰器做一定的处理，使__init__只调用一次
    setattr(cls, "__init__", run_only_once(cls.__init__))
    return cls

In [111]:
@SingletonX   
class P:
    a = 12

    def __init__(self, name=None, age=None):
        print("__init__!")
        self.name = name
        self.age = age
#         self.t = Person.t

def create_p():
    person1 = P('lucky', 24)
    person2 = P('lau', 26)
    person3 = P()
    print(id(person1))
    print(id(person2))
    print(id(person3))

create_p()

meow: 12
__init__!
139947553174024
139947553174024
139947553174024


In [112]:
class Pl(P):
    a = 24
    
p = Pl()
print(p.a)

12
